In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import requests
from bs4 import BeautifulSoup

import time
from datetime import datetime

import pandas as pd

In [2]:
def crawling_youtuber_video_link(path, youtuber, num_end=10):
    """해당 유튜버의 영상 링크들을 가져오기
    
    path : 크롬드라이버 주소
    youtuber : 유튜버의 홈 링크
    num_end : 동영상 탭에서 end 키를 누르는 횟수 => 비디오 링크를 얼마나 가져올 것인가
    
    가져오는 변수를 좀 추가해야되나??
    여기서 더 안가져오고 동영상 링크 들어가면 거기서 많이 가져오면 될 것 같기도 하고
    """
    driver = webdriver.Chrome(path)
    driver.maximize_window() # 창 크기 최대화
    
    driver.get(youtuber) # 해당 링크로 이동
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="tabsContent"]/paper-tab[2]/div').click() # 버튼 클릭
    
    # end 키를 눌러 페이지 맨 밑으로 이동
    for i in range(num_end):
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(5)
        
    dom = driver.page_source
    soup = BeautifulSoup(dom, 'html.parser')
    link_source = soup.select("div#details") # 영상들이 있는 부분의 소스 추출
    
    video_link_list = []
    
    for link in link_source:
        title = link.find('a')['title']
        href = link.find('a')['href']
        video_link_list.append([datetime.now().strftime("%Y/%m/%d %H:%M:%S"), title, href])
    
    driver.close()
    
    youtuber_video_link = pd.DataFrame(video_link_list, columns=['datetime', 'title', 'url'])
    
    return youtuber_video_link

In [3]:
start = time.time()

ytn_video_list = crawling_youtuber_video_link(path='chromedriver.exe',
                                              youtuber='https://www.youtube.com/user/ytnnews24/featured',
                                              num_end=20)

print('돌아가는데 걸린 시간 : {}분 {}초'.format(round((time.time() - start) // 60), round((time.time() - start) % 60)))

돌아가는데 걸린 시간 : 2분 1초


In [4]:
len(ytn_video_list)

595

In [5]:
ytn_video_list.head(5)

,datetime,title,url
0,2020/07/29 14:00:09,[날씨] 광주· 대전 호우경보 ...내일까지 국지성 호우 / YTN,/watch?v=djMlBTqgoSk
1,2020/07/29 14:00:09,부동산 입법 속도내지만...시장은 여전히 '?' / YTN,/watch?v=MB5mHk2a1LI
2,2020/07/29 14:00:09,"[뉴스앤이슈] 與, 부동산법안 '속전속결'...통합, 반발 속 '속수무책' / YTN",/watch?v=M0OeJIvakMQ
3,2020/07/29 14:00:09,美 '반중 연대' 강화...中 '사법 공조' 중단 / YTN,/watch?v=uzMjwfMDdns
4,2020/07/29 14:00:09,[취재N팩트] 한미 미사일지침 개정...민·군에 어떤 효과? / YTN,/watch?v=K3k0WqujRdo


In [6]:
ytn_video_list.tail(5)

,datetime,title,url
590,2020/07/29 14:00:09,'불기소 권고' 한 달 지났지만...'이재용 기소 여부' 결론 이달 넘길 듯 / YTN,/watch?v=fZa1qVD-voI
591,2020/07/29 14:00:09,"빌 게이츠 ""SK바이오사이언스, 백신 성공하면 내년 6월부터 연간 2억 개 생산 가...",/watch?v=OWEZcGJ48mk
592,2020/07/29 14:00:09,중앙재난안전대책본부 브리핑 (7월 25일) / YTN,/watch?v=ldibnLo6ynU
593,2020/07/29 14:00:09,"시민단체, 중복 맞아 개 식용·도살 금지법 제정 촉구 / YTN",/watch?v=EO6654EHc1M
594,2020/07/29 14:00:09,"울산 앞바다에 승용차 추락, 운전자 구조 / YTN",/watch?v=cgSxSx8MZyA


동영상의 링크를 이런식으로 가져와봤어요.

이제 해당 링크를 들어가서 필요한 정보들을 긁어오는 함수를 추가할 예정 => 셀레니움을 이용해서 하는 방법은 page_source에서 찾는건 다 했는데 아직 깔끔하게 만들지를 못함

필요한 정보 : 영상 제목, 링크, 유튜버, 게시 날짜, 좋아요 수, 싫어요 수, 영상 정보, 영상의 자동 생성 스크립트, 댓글의 정렬 기준을 인기 댓글 순으로 했을 때 상위 댓글 10개

그리고 만약에 유튜브에서 데이터를 10만 개를 가져온다고 했을 때 selenium을 가지고 하기엔 힘들다고 생각이 돼서 안쓰고 하는 방법을 더 찾아보도록 하겠습니다.

셀레니움을 사용한 이유는 유튜버 사이트 특성 상 버튼을 누른다거나, 스크롤을 움직여야 html page source가 추가되기 때문이었는데 다른 라이브러리 사용했을때에도 그렇게 되는지 아닌지 잘 모르겠네요. 시도해보겠습니다.